In [2]:
# load binary file
import numpy as np
import time
import suite2p, register, dcnv, celldetect

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'diameter': 14,
    'tau': 2.,
      }

# get defaults options
ops0 = suite2p.default_ops()

# combine with user options
ops = {**ops0, **ops} 

2.0

In [3]:
# copy tiff to a binary
ops1 = register.tiff_to_binary(ops)
# register tiff
ops1 = register.register_binary(ops1)

In [14]:
import imp
imp.reload(celldetect)
U,sdmov      = celldetect.getSVDdata(ops1[0])

73.36479115486145
82.30986762046814
93.87892723083496
99.66246008872986


In [21]:
data = np.random.randn(1000,1000,1000)

In [23]:
data = data.reshape((1000, -1))